# Enfrentando o Dragão
**Mural de Quests 04 - Jardim do Palácio**

**Disciplina:** Aprendizado de Máquina

**Professor:** Daniel R. Cassar

**Alunas:** Lília Gavazza, Lívia Aragão, Sophia Alves

Esse é o primeiro teste dos cinco modelos a serem aplicados no dataset escolhido.

In [2]:
# Importando bibliotecas e funções necessárias
import pandas as pd
import numpy as np
import matplotlib as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier

from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from optuna import create_study
from sklearn.model_selection import cross_val_score

# [Modelos]
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

C:\Users\livia25013\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Carregando dados

In [3]:
colunas = ["Crystal System", "Sites", "Energy Above Hull", 
           "Formation Energy", "Volume", "Density", "Band Gap", "Is Gap Direct", "Is Metal",
           "Magnetic Ordering"]

nome_do_arquivo = "magnetismo_Co(in).csv"
magnetic_co = pd.read_csv(nome_do_arquivo)

magnetic_co = magnetic_co.dropna() # retirar linhas com valores vazios 
magnetic_co = magnetic_co.reindex(colunas, axis = 1)

label_encoder = LabelEncoder()
magnetic_co["Magnetic Ordering"] = label_encoder.fit_transform(magnetic_co["Magnetic Ordering"])

print("Mapeamento do Label Encoder:")
for i, classe in enumerate(label_encoder.classes_):
    print(f"  {classe} -> {i}")

TAMANHO_TESTE = 0.1 
SEMENTE_ALEATORIA = 73

indices = magnetic_co.index
indices_treino, indices_teste = train_test_split(
    indices, test_size = TAMANHO_TESTE, random_state = SEMENTE_ALEATORIA)

# Definição dos DataFrames com os dados de treino e de teste
treino_mco = magnetic_co.loc[indices_treino]
teste_mco = magnetic_co.loc[indices_teste]

FEATURES = ["Crystal System", "Sites", "Energy Above Hull", "Is Metal", "Is Gap Direct",
           "Formation Energy", "Volume", "Density", "Band Gap"]
TARGET = ["Magnetic Ordering"]

X_treino_df = treino_mco.reindex(FEATURES, axis = 1)
X_teste_df = teste_mco.reindex(FEATURES, axis = 1)

# Identificar colunas categóricas
categorical_cols = X_treino_df.select_dtypes(include=['object', 'bool', 'category']).columns.tolist()
numerical_cols = X_treino_df.select_dtypes(include=[np.number]).columns.tolist()

# Separação e seleção dos valores de features e target por DF
X_treino = treino_mco.reindex(FEATURES, axis = 1)
y_treino = treino_mco.reindex(TARGET, axis = 1).values.ravel()

X_teste = teste_mco.reindex(FEATURES, axis = 1)
y_teste = teste_mco.reindex(TARGET, axis = 1).values.ravel()

Mapeamento do Label Encoder:
  AFM -> 0
  FM -> 1
  FiM -> 2
  NM -> 3


In [5]:
preprocessador = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ])

#### Baseline 
Esse é o modelo base, ele apenas prevê a classe mais frequente para todos os dados. Não existe necessidade, portanto de avaliar o melhor conjunto de hiperparâmetro, pois eles não existem nesse caso.

In [6]:
pipeline_bsl = Pipeline([
    ("prep", preprocessador),
    ("dummy", DummyClassifier(strategy='stratified', random_state=SEMENTE_ALEATORIA))
])
    
pipeline_bsl.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto1 = pipeline_bsl.predict(X_teste)

acuracia_bsl = accuracy_score(y_verdadeiro, y_previsto1)
print(f"Acurácia - Baseline: {round((acuracia_bsl)*100, 2)}%")

Acurácia - Baseline: 40.74%


Esse desempenho será o nosso norte quanto a real capacidade de generalização dos próximos algoritmos, pensando que quanto maior e mais distante dessa base, melhor é o modelo induzido.

### Otimização de Hiperparâmetros com `optuna`
O `optuna` é um módulo de python que permite uma otimização de hiperparâmetros por uma espécie de "conjunto de tentativas". O `trial` é um objeto dele que sorteia e sugere parâmetros para o modelo, criando seu espaço de busca de hiperparâmetros. 

Disso, é definida a função objetivo para medir, reforçada por uma validação cruzada k-fold, a métrica sob análise, nesse caso, a acurácia. Efetivamente o que ocorre é que o conjunto de hiperparâmetros sugerido no passo anterior é avaliado por essa estratégia, que devolve a estimativa de desempenho dele para guiar e comparar com as próximas escolhas. Isso é feito por uma subdivisão dos dados de treino em $k$ subconjuntos, que serão utilizados como dados de teste uma vez. A média das $k$ estimativas de desempenho obtidas é mais robusta em determinar a performance do modelo "induzido".

Por fim, a criação do objeto de estudo define o otimizador desse problema de maximização que será rodado com módulo `optimize` um determinado número de tentativas (100). Os melhores parâmetros encontrados serão os utilizados para realmente induzir o modelo e calcular sua acurácia

#### k-NN Classificador
Esse modelo se baseia na moda de classes de n dos seus vizinhos para prever um dado desconhecido. Seus principais hiperparâmetros são:
- `n_neighbors`: o número de vizinhos a serem considerados na análise da moda;
- `weights`: como as classes dos vizinhos influenciam na previsão, de forma uniforme com todos tendo o meso peso ou de forma guiada pela distância, os mais próximos tem maior peso;
- `p`: o tipo de distância considerada na determinação dos vizinhos mais próximos.

Também será analisada a influência/necessidade da normalização dos dados.

In [10]:
def instancia_kNN(trial):
    parametros_kNN = {
        "n_neighbors": trial.suggest_int("num_vizinhos", 1, 100, log=True),
        "weights": trial.suggest_categorical("pesos", ["uniform", "distance"]),
        "p": trial.suggest_float("potencia_minkowski", 1, 2),
        "n_jobs": -1 # processamento
    }
    
    modelo_kNN = Pipeline([
            ("pre", preprocessador),
            ("kNN", KNeighborsClassifier(**parametros_kNN))
    ])

    return modelo_kNN           

def func_obj_kNN(trial, X, y, n_folds):
    
    modelo = instancia_kNN(trial)

    metricas_kNN = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=n_folds,
        error_score='raise'
    )

    return metricas_kNN.mean()

NOME_DO_ESTUDO = "kNN"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:01:40,283] Using an existing study with name 'kNN' instead of creating a new one.


In [11]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_kNN(trial):
    return func_obj_kNN(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_kNN, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:01:42,527] Trial 100 finished with value: 0.5745748299319728 and parameters: {'num_vizinhos': 2, 'pesos': 'distance', 'potencia_minkowski': 1.6168699521398098}. Best is trial 100 with value: 0.5745748299319728.
[I 2025-11-04 00:01:42,701] Trial 101 finished with value: 0.5745748299319728 and parameters: {'num_vizinhos': 1, 'pesos': 'distance', 'potencia_minkowski': 1.6259371822797064}. Best is trial 100 with value: 0.5745748299319728.
[I 2025-11-04 00:01:42,887] Trial 102 finished with value: 0.5745748299319728 and parameters: {'num_vizinhos': 2, 'pesos': 'distance', 'potencia_minkowski': 1.612293772173357}. Best is trial 100 with value: 0.5745748299319728.
[I 2025-11-04 00:01:43,085] Trial 103 finished with value: 0.5575680272108844 and parameters: {'num_vizinhos': 5, 'pesos': 'distance', 'potencia_minkowski': 1.617954244562084}. Best is trial 100 with value: 0.5745748299319728.
[I 2025-11-04 00:01:43,251] Trial 104 finished with value: 0.5619047619047619 and paramete

In [12]:
melhor_trial_kNN = objeto_de_estudo.best_trial

modelo_kNN = instancia_kNN(melhor_trial_kNN)
modelo_kNN.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto2 = modelo_kNN.predict(X_teste)

f1_kNN = f1_score(y_verdadeiro, y_previsto2, average = "weighted")

print(f"Melhor trial: {melhor_trial_kNN.number} - Parâmetros: {melhor_trial_kNN.params}")
print(f"Acurácia - kNN: {f1_kNN}")

Melhor trial: 127 - Parâmetros: {'num_vizinhos': 7, 'pesos': 'distance', 'potencia_minkowski': 1.4254359569302124}
Acurácia - kNN: 0.8282828282828283


#### Árvore de Decisão
As previsões são feitas por meio da sudivisão dos dados de acordo com limiares de um determinado critério. Seus principais hiperparâmetros são:
- `max_leaf_nodes`: número máximo de vértices folha;
- `max_depth`: máxima profundidade da árvore de decisão gerada;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha.

Novamente, a possibilidade de normalização foi considerada.

In [13]:
def instancia_ad(trial):
    parametros_ad = {
        "max_leaf_nodes": trial.suggest_int("num_maximo_nos", 2, 20),
        "max_depth": trial.suggest_categorical("profundidade_maxima", [None, 3, 6, 9, 12, 18, 21, 24]),
        "min_samples_split": trial.suggest_int("min_exemplos_vcondicional", 2, 20),
        "min_samples_leaf": trial.suggest_int("min_exemplos_vfolha", 1, 20),
        "random_state": 73 #reprotibilidade
    }

    modelo_ad = Pipeline([
            ("pre", preprocessador),
            ("arv_dec", DecisionTreeClassifier(**parametros_ad))
    ])

    return modelo_ad

def func_obj_ad(trial, X, y, n_folds):
    modelo = instancia_ad(trial)

    metricas_ad = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        
        cv=n_folds
    )

    return metricas_ad.mean()

NOME_DO_ESTUDO = "arvore_de_decisao"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:02:30,093] A new study created in RDB with name: arvore_de_decisao


In [14]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_ad(trial):
    return func_obj_ad(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_ad, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:02:34,399] Trial 0 finished with value: 0.4915816326530612 and parameters: {'num_maximo_nos': 3, 'profundidade_maxima': 9, 'min_exemplos_vcondicional': 5, 'min_exemplos_vfolha': 5}. Best is trial 0 with value: 0.4915816326530612.
[I 2025-11-04 00:02:34,568] Trial 1 finished with value: 0.4754251700680273 and parameters: {'num_maximo_nos': 12, 'profundidade_maxima': 9, 'min_exemplos_vcondicional': 15, 'min_exemplos_vfolha': 18}. Best is trial 0 with value: 0.4915816326530612.
[I 2025-11-04 00:02:34,710] Trial 2 finished with value: 0.4996598639455782 and parameters: {'num_maximo_nos': 10, 'profundidade_maxima': None, 'min_exemplos_vcondicional': 7, 'min_exemplos_vfolha': 16}. Best is trial 2 with value: 0.4996598639455782.
[I 2025-11-04 00:02:34,862] Trial 3 finished with value: 0.4915816326530612 and parameters: {'num_maximo_nos': 3, 'profundidade_maxima': 21, 'min_exemplos_vcondicional': 14, 'min_exemplos_vfolha': 1}. Best is trial 2 with value: 0.4996598639455782.
[I

In [15]:
melhor_trial_ad = objeto_de_estudo.best_trial

modelo_ad = instancia_ad(melhor_trial_ad)
modelo_ad.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto3 = modelo_ad.predict(X_teste)

acuracia_ad = accuracy_score(y_verdadeiro, y_previsto3)

print(f"Melhor trial: {melhor_trial_ad.number} - Parâmetros: {melhor_trial_ad.params}")
print(f"Acurácia - Árvore de Decisão: {acuracia_ad}")

Melhor trial: 7 - Parâmetros: {'num_maximo_nos': 5, 'profundidade_maxima': 3, 'min_exemplos_vcondicional': 6, 'min_exemplos_vfolha': 5}
Acurácia - Árvore de Decisão: 0.6666666666666666


#### Floresta Aleatória
Nesse algoritmos são projetadas vários conjuntos de árvores de decisão, tentando criar um modelo de maior variância e menor viés. Seus principais hiperparâmetros são:
- `n_estimators`: número de árvores por comitê;
- `min_samples_split`: número mínimo de exemplos para gerar uma divisão condicional deles;
- `min_samples_leaf`: número mínimo de exemplos em um vértice folha;
- `max_features`: fração de atributos usada em cada _split_ das árvores;
- `bootstrap`: se ocorre uma amostragem do tipo _bootstrap_ dos exemplos para treinar as árvores.

In [16]:
def instancia_fa(trial):
    parametros_fa = {
        "n_estimators": trial.suggest_int("num_arvores", 50, 500, log=True),
        "min_samples_split": trial.suggest_int("min_exemplos_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_exemplos_folha", 1, 10),
        "max_features": trial.suggest_float("num_max_atributos", 0, 1),
        "bootstrap": trial.suggest_categorical("bootstrap", [True, False]),
        "n_jobs": -1, #processamento
        "random_state": 73 #reprodutibilidade
    }

    modelo_fa = Pipeline([
            ("pre", preprocessador),
            ("flo_ale", RandomForestClassifier(**parametros_fa))
    ])      
    

    return modelo_fa

def func_obj_fa(trial, X, y, num_folds):
    modelo = instancia_fa(trial)

    metricas_fa = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_fa.mean()

NOME_DO_ESTUDO = "floresta_aleatoria"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:03:52,416] A new study created in RDB with name: floresta_aleatoria


In [17]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_fa(trial):
    return func_obj_fa(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_fa, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:03:55,885] Trial 0 finished with value: 0.5537414965986394 and parameters: {'num_arvores': 104, 'min_exemplos_split': 3, 'min_exemplos_folha': 10, 'num_max_atributos': 0.20694286480985813, 'bootstrap': True}. Best is trial 0 with value: 0.5537414965986394.
[I 2025-11-04 00:03:56,462] Trial 1 finished with value: 0.5284863945578231 and parameters: {'num_arvores': 88, 'min_exemplos_split': 7, 'min_exemplos_folha': 6, 'num_max_atributos': 0.9981079439558666, 'bootstrap': False}. Best is trial 0 with value: 0.5537414965986394.
[I 2025-11-04 00:03:57,466] Trial 2 finished with value: 0.561734693877551 and parameters: {'num_arvores': 200, 'min_exemplos_split': 6, 'min_exemplos_folha': 7, 'num_max_atributos': 0.3402019516126312, 'bootstrap': False}. Best is trial 2 with value: 0.561734693877551.
[I 2025-11-04 00:03:58,187] Trial 3 finished with value: 0.5619047619047619 and parameters: {'num_arvores': 78, 'min_exemplos_split': 6, 'min_exemplos_folha': 9, 'num_max_atributos': 

In [18]:
melhor_trial_fa = objeto_de_estudo.best_trial

modelo_fa = instancia_fa(melhor_trial_fa)
modelo_fa.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto4 = modelo_fa.predict(X_teste)

acuracia_fa = accuracy_score(y_verdadeiro, y_previsto4)

print(f"Melhor trial: {melhor_trial_fa.number} - Parâmetros: {melhor_trial_fa.params}")
print(f"Acurácia - Floresta Aleatória: {acuracia_fa:.2f}")

Melhor trial: 91 - Parâmetros: {'num_arvores': 262, 'min_exemplos_split': 2, 'min_exemplos_folha': 1, 'num_max_atributos': 0.20392435292650418, 'bootstrap': True}
Acurácia - Floresta Aleatória: 0.89


#### Naive Bayes Gaussiano
Esse algoritmo assume _features_ independentes e faz previsões baseadas nas probailidades das classes assumindo as _features_ em distribuição gaussiana e o Teorema de Bayes. Seu principal hiperparâmetro é:
- `var_smoothing`: controla a variância que afeta as distribuições das _features_, para aumentar a estabilidade da previsão.

In [19]:
def instancia_nbg(trial):
    parametros_nbg = {
        "var_smoothing": trial.suggest_float("var_smoothing", 1e-11, 1e-1, log=True)
    }

    modelo_nbg = Pipeline([
        ("pre", preprocessador),
        ("nbg", GaussianNB(**parametros_nbg))
    ])
    
    return modelo_nbg

def func_obj_nbg(trial, X, y, num_folds):
    modelo = instancia_nbg(trial)

    metricas_nbg = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_nbg.mean()

NOME_DO_ESTUDO = "naive_bayes_gaussiano"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:06:22,560] Using an existing study with name 'naive_bayes_gaussiano' instead of creating a new one.


In [20]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_nbg(trial):
    return func_obj_nbg(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_nbg, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:06:25,343] Trial 100 finished with value: 0.4172619047619047 and parameters: {'var_smoothing': 3.6746031625342624e-07}. Best is trial 12 with value: 0.6030612244897958.
[I 2025-11-04 00:06:25,430] Trial 101 finished with value: 0.5785714285714285 and parameters: {'var_smoothing': 0.058875765480452195}. Best is trial 12 with value: 0.6030612244897958.
[I 2025-11-04 00:06:25,541] Trial 102 finished with value: 0.5661564625850339 and parameters: {'var_smoothing': 0.03128436289737384}. Best is trial 12 with value: 0.6030612244897958.
[I 2025-11-04 00:06:25,640] Trial 103 finished with value: 0.5621598639455783 and parameters: {'var_smoothing': 0.09831410730109451}. Best is trial 12 with value: 0.6030612244897958.
[I 2025-11-04 00:06:25,739] Trial 104 finished with value: 0.5496598639455782 and parameters: {'var_smoothing': 0.01487064812093953}. Best is trial 12 with value: 0.6030612244897958.
[I 2025-11-04 00:06:25,841] Trial 105 finished with value: 0.5785714285714285 and

In [21]:
melhor_trial_nbg = objeto_de_estudo.best_trial

modelo_nbg = instancia_nbg(melhor_trial_nbg)
modelo_nbg.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto5 = modelo_nbg.predict(X_teste)

acuracia_nbg = accuracy_score(y_verdadeiro, y_previsto5)

print(f"Melhor trial: {melhor_trial_nbg.number} - Parâmetros: {melhor_trial_nbg.params}")
print(f"Acurácia - Naive Bayes Gaussiano: {acuracia_nbg:.2f}")

Melhor trial: 12 - Parâmetros: {'var_smoothing': 0.0757065689985383, 'normalizacao': False}
Acurácia - Naive Bayes Gaussiano: 0.74


#### Support Vector Classification - SVC
Nesse modelo, a multiclassficação é subdividida em problemas de classificação binária. Em cada um deles, procura o hiperplano que melhora separa as diferentes classes. Seus principais hiperparâmetros são:
- `kernel`: método de separação dos grupos;
- `C`: parâmetro de regularização;
- `gamma`: controla a influência de cada ponto de treino em _kernels_ não lineares;
- `max_iter`:

In [22]:
def instancia_svc(trial):
    parametros_svc = {
        "kernel": trial.suggest_categorical("kernel", ["rbf", "poly", "sigmoid"]),
        "C": trial.suggest_float("C", 0.01, 100, log=True),
        "gamma": trial.suggest_categorical("gamma", ["scale", "auto"]),
        "max_iter": 5000,
        "random_state": 73 #reprodutibilidade
    }

    modelo_svc = Pipeline([
        ("pre", preprocessador),
        ("svc", SVC(**parametros_svc))
    ])

    return modelo_svc  



def func_obj_svc(trial, X, y, num_folds):
    modelo = instancia_svc(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "svc"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:06:48,729] Using an existing study with name 'svc' instead of creating a new one.


In [23]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_svc(trial):
    return func_obj_svc(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_svc, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:06:51,023] Trial 100 finished with value: 0.45901360544217684 and parameters: {'kernel': 'poly', 'C': 3.508690538377317, 'gamma': 'auto'}. Best is trial 34 with value: 0.6318027210884354.
[I 2025-11-04 00:06:51,172] Trial 101 finished with value: 0.5870748299319727 and parameters: {'kernel': 'poly', 'C': 11.416879059638921, 'gamma': 'auto'}. Best is trial 34 with value: 0.6318027210884354.
[I 2025-11-04 00:06:51,290] Trial 102 finished with value: 0.5911564625850341 and parameters: {'kernel': 'poly', 'C': 10.023552863510568, 'gamma': 'auto'}. Best is trial 34 with value: 0.6318027210884354.
[I 2025-11-04 00:06:51,526] Trial 103 finished with value: 0.5375850340136055 and parameters: {'kernel': 'poly', 'C': 5.2889707375542745, 'gamma': 'auto'}. Best is trial 34 with value: 0.6318027210884354.
[I 2025-11-04 00:06:51,670] Trial 104 finished with value: 0.5912414965986394 and parameters: {'kernel': 'poly', 'C': 7.20014245097232, 'gamma': 'auto'}. Best is trial 34 with valu

In [24]:
melhor_trial_svc = objeto_de_estudo.best_trial

modelo_svc = instancia_svc(melhor_trial_svc)
modelo_svc.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_svc.predict(X_teste)

acuracia_svc = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_svc.number} - Parâmetros: {melhor_trial_svc.params}")
print(f"Acurácia - SVC: {acuracia_svc}")

Melhor trial: 34 - Parâmetros: {'kernel': 'rbf', 'C': 8.134234798526416, 'gamma': 'auto', 'normalizacao': False}
Acurácia - SVC: 0.8148148148148148


#### Regressão Logística

In [25]:
def instancia_lr(trial):
    parametros_lr = {
        'penalty':  trial.suggest_categorical("penalty", ['l1','l2']),
        'C': trial.suggest_float("C", 1e-4, 1e4, log=True),
        'solver': "saga",
        'max_iter': trial.suggest_int("max_iter", 100, 5000, log=True),
        'tol': 1e-2,
        'random_state': 73
}

    modelo_lr = Pipeline([
        ("pre", preprocessador),
        ("log_reg", LogisticRegression(**parametros_lr))
    ])

    return modelo_lr


def func_obj_lr(trial, X, y, num_folds):
    modelo = instancia_lr(trial)

    metricas_svc = cross_val_score(
        modelo,
        X,
        y,
        scoring="accuracy",
        cv=num_folds,
    )

    return metricas_svc.mean()

NOME_DO_ESTUDO = "lr"

objeto_de_estudo = create_study(
    direction="maximize",
    study_name=NOME_DO_ESTUDO,
    storage=f"sqlite:///{NOME_DO_ESTUDO}.db",
    load_if_exists=True,
)

[I 2025-11-04 00:07:11,353] A new study created in RDB with name: lr


In [26]:
NUM_FOLDS = 5
NUM_TENTATIVAS = 100

def func_obj_parcial_lr(trial):
    return func_obj_lr(trial, X_treino, y_treino, NUM_FOLDS)

objeto_de_estudo.optimize(func_obj_parcial_lr, n_trials=NUM_TENTATIVAS)

[I 2025-11-04 00:07:15,288] Trial 0 finished with value: 0.37193877551020404 and parameters: {'penalty': 'l1', 'C': 0.025800758423157103, 'max_iter': 1204}. Best is trial 0 with value: 0.37193877551020404.
[I 2025-11-04 00:07:15,473] Trial 1 finished with value: 0.5827380952380953 and parameters: {'penalty': 'l2', 'C': 25.29797755459552, 'max_iter': 1467}. Best is trial 1 with value: 0.5827380952380953.
[I 2025-11-04 00:07:15,671] Trial 2 finished with value: 0.37193877551020404 and parameters: {'penalty': 'l1', 'C': 0.03789440916403047, 'max_iter': 1064}. Best is trial 1 with value: 0.5827380952380953.
[I 2025-11-04 00:07:15,838] Trial 3 finished with value: 0.5785714285714285 and parameters: {'penalty': 'l1', 'C': 124.26328194310561, 'max_iter': 490}. Best is trial 1 with value: 0.5827380952380953.
[I 2025-11-04 00:07:15,965] Trial 4 finished with value: 0.37193877551020404 and parameters: {'penalty': 'l1', 'C': 0.014356448691009624, 'max_iter': 218}. Best is trial 1 with value: 0.58

In [27]:
melhor_trial_lr = objeto_de_estudo.best_trial

modelo_lr = instancia_lr(melhor_trial_lr)
modelo_lr.fit(X_treino, y_treino)

y_verdadeiro = y_teste
y_previsto7 = modelo_lr.predict(X_teste)

acuracia_lr = accuracy_score(y_verdadeiro, y_previsto7)

print(f"Melhor trial: {melhor_trial_lr.number} - Parâmetros: {melhor_trial_lr.params}")
print(f"Acurácia - SVC: {acuracia_lr}")

Melhor trial: 5 - Parâmetros: {'penalty': 'l2', 'C': 0.20746197169173597, 'max_iter': 277}
Acurácia - SVC: 0.8518518518518519


### Referências
1. `sckit-learn` - Geração de Datasets. Disponível em: [https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators](https://scikit-learn.org/stable/datasets/sample_generators.html#sample-generators)
2. `sckit-learn` - Validação Cruzada. Disponível em: [https://scikit-learn.org/stable/modules/cross_validation.html](https://scikit-learn.org/stable/modules/cross_validation.html)
3. `sckit-learn` - Naive Bayes. Disponível em: [https://scikit-learn.org/stable/modules/naive_bayes.html](https://scikit-learn.org/stable/modules/naive_bayes.html)
4. BAELDUNG. Multiclass Classification Using Support Vector Machines | Baeldung on Computer Science. Disponível em: [https://www.baeldung.com/cs/svm-multiclass-classification](https://www.baeldung.com/cs/svm-multiclass-classification)
5. `sckit-learn` - `SVC`. Disponível em: [https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html](https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html) 
6. CASSAR, Daniel. Materiais de Aula.